<a href="https://colab.research.google.com/github/lkaykymar/kayky_martins/blob/main/codigo_de_similaridade_cosseno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Recomendação Por Título**

In [5]:
import pandas as pd
import re
import unicodedata
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import math
import nltk
from nltk.corpus import stopwords

# Baixar stopwords
nltk.download('stopwords')

# Lista de stopwords em português
stop_words = set(stopwords.words('portuguese'))

# Função para limpar os títulos das músicas
def preprocess_song_name(song_name):
    if pd.isna(song_name):
        return ""

    title = re.sub(r"[\[\]']", "", song_name)
    title = re.sub(r"[^\w\s]", "", title)
    title = title.lower()
    title = ''.join(
        char for char in unicodedata.normalize('NFD', title)
        if unicodedata.category(char) != 'Mn'
    )

    # Remove stop words
    words = title.split()
    filtered_words = [word for word in words if word not in stop_words]

    return " ".join(filtered_words)

# Função de recomendação
def recommend_songs(user_input, songs_df):
    user_input = preprocess_song_name(user_input)
    user_vector = np.zeros(len(unique_words))

    for word in user_input.split():
        if word in word_index:
            user_vector[word_index[word]] = 1

    similarities = cosine_similarity([user_vector], word_bow.values)[0]
    angles = [round(math.degrees(math.acos(sim)), 1) if -1 <= sim <= 1 else None for sim in similarities]

    songs_df['similarity'] = similarities
    songs_df['angle_degrees'] = angles

    recommended_songs = songs_df.sort_values(by='similarity', ascending=False).head(10)
    return recommended_songs[['song_name', 'artist']]

# Carregando o dataset


data_path = '/content/bossa_nova_songs_portugues.xlsx'
songs_df = pd.read_excel(data_path)


# Limpando os nomes das músicas
songs_df['cleaned_song_name'] = songs_df['song_name'].apply(preprocess_song_name)

# Criando o índice de palavras únicas
unique_words = sorted(set(" ".join(songs_df['cleaned_song_name']).split()))
word_index = {word: idx for idx, word in enumerate(unique_words)}

# Criando a matriz Bag of Words
word_bow = pd.DataFrame(0, index=songs_df.index, columns=unique_words)
for idx, song in enumerate(songs_df['cleaned_song_name']):
    for word in song.split():
        if word in word_bow.columns:
            word_bow.loc[idx, word] = 1

# Entrada do usuário e recomendação
user_input = str(input("Digite o nome da música para recomendar: "))
recommendations = recommend_songs(user_input, songs_df)

# Exibindo somente nomes e artistas recomendados
for idx, row in recommendations.iterrows():
    print(f"{row['song_name']} - {row['artist']}")




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Digite o nome da música para recomendar: girassol
 O Girassol   -   Vinicius de Moraes 
 Catavento e Girassol   -   Sérgio Mendes 
 Tomara   -   Alaíde Costa 
 Tempo Calado   -   Alaíde Costa 
 Quem Sou Eu?   -   Alaíde Costa 
 Diariamente   -   Alaíde Costa 
 Coração   -   Alaíde Costa 
 Absinto   -   Alaíde Costa 
 Amigo Amado   -   Alaíde Costa 
 Onde Está Você   -   Alaíde Costa 


**Recomendação por Letra**

In [9]:
import pandas as pd
import re
import unicodedata
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math
import nltk
from nltk.corpus import stopwords

# Baixando a lista de stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))  # Utilizando stopwords em português

def preprocess_lyrics(lyrics):
    if pd.isna(lyrics):
        return ""
    lyrics = lyrics.lower()  # Transforma tudo em minúsculo
    lyrics = re.sub(r'[^\w\s]', '', lyrics)  # Remove pontuações
    lyrics = ''.join(char for char in unicodedata.normalize('NFD', lyrics) if unicodedata.category(char) != 'Mn')  # Remove acentos
    lyrics = ' '.join(word for word in lyrics.split() if word not in stop_words)  # Remove stopwords
    return lyrics

def recommend_songs(user_input, songs_df):
    user_input = preprocess_lyrics(user_input)  # Pré-processa a entrada do usuário

    all_lyrics = songs_df['cleaned_song_lyrics'].tolist() + [user_input]  # Junta as letras das músicas com a entrada do usuário

    vectorizer = CountVectorizer(binary=True)  #  (bag of words)
    bow_matrix = vectorizer.fit_transform(all_lyrics)

    similarities = cosine_similarity(bow_matrix[-1], bow_matrix[:-1])[0]  # Calcula a similaridade de cosseno
    angles = [round(math.degrees(math.acos(sim)), 1) if -1 <= sim <= 1 else None for sim in similarities]

    songs_df['similarity'] = similarities  # Adiciona a similaridade no dataframe
    songs_df['angle_degrees'] = angles  # Adiciona o ângulo em graus no dataframe

    recommended_songs = songs_df.sort_values(by='similarity', ascending=False).head(10)  # Seleciona as 10 músicas mais similares
    return recommended_songs[['song_name', 'artist', 'song_lyrics']]  # Retorna as colunas desejadas

# Carregamento dos dados
data_path = '/content/bossa_nova_songs_portugues.xlsx'
songs_df = pd.read_excel(data_path)

# Limpeza das letras das músicas
songs_df['cleaned_song_lyrics'] = songs_df['song_lyrics'].apply(preprocess_lyrics)

# Entrada do usuário
user_input = str(input("Digite a letra ou parte da letra da música para recomendar: "))
recommendations = recommend_songs(user_input, songs_df)

# Limite de caracteres para exibir da letra
MAX_LETRA = 130

# Exibindo as recomendações (título, artista e letra completa limitada)
print("Músicas recomendadas:")
for index, row in recommendations.iterrows():
    letra = row['song_lyrics']
    if len(letra) > MAX_LETRA:
        letra_exibida = letra[:MAX_LETRA] + "..."
    else:
        letra_exibida = letra

    print(f"Título: {row['song_name']}")
    print(f"Artista: {row['artist']}")
    print(f"Letra (trecho):\n{letra_exibida}\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Digite a letra ou parte da letra da música para recomendar: girassol
Músicas recomendadas:
Título:  O Girassol  
Artista:   Vinicius de Moraes 
Letra (trecho):
  Sempre que o sol  Pinta de anil  Todo o céu  O girassol  Fica um gentil  Carrossel  Roda, roda, roda  Carrossel  Roda, roda, rod...

Título:  Catavento e Girassol  
Artista:   Sérgio Mendes 
Letra (trecho):
  Meu catavento tem dentro O que há do lado de fora do teu girassol Entre o escancaro e o contido Eu te pedi sustenido E você riu ...

Título:  Cântico  
Artista:   Vinicius de Moraes 
Letra (trecho):
  Não, tu não és um sonho, és a existência Tens carne, tens fadiga e tens pudor No calmo peito teu. Tu és a estrela Sem nome, és a...

Título:  Absinto  
Artista:   Alaíde Costa 
Letra (trecho):
  Hoje a coisa mais firme que creio É que eu te odeio E é esse o ódio que eu mais choro Eu te adoro Eu te adoro Eu bebo essas água...

Título:  Tempo Calado  
Artista:   Alaíde Costa 
Letra (trecho):
  Ah! Se o esquecer me for negado E